In [1]:
import sys
sys.path.append('../library/src')
import rnnbuilder as rb
import torch
from rnnbuilder import snn, rnn, nn

In [2]:

seq = 8
batch = 32
inp_shape = (1,92,76)
example = torch.rand((seq,batch)+inp_shape)
FRAME_STACK = 4
N_OUT = 3

CONV_NEURON = rb.snn.LIF()#rb.nn.ReLU()#rb.snn.Discontinuous()

conv_stack = rb.Sequential(
    rb.rnn.TempConvPlus2d(out_channels=32, kernel_size=8, stride=4, time_kernel_size=FRAME_STACK), CONV_NEURON,
    rb.nn.Conv2d(out_channels=64, kernel_size=4, stride=2), CONV_NEURON,
    rb.nn.Conv2d(out_channels=64, kernel_size=3, stride=1), CONV_NEURON)

ll_rsnn = rb.Network()
ll_rsnn.output = rb.Placeholder()
ll_rsnn.output = ll_rsnn.input.stack(ll_rsnn.output).apply(rb.nn.Linear(512), rb.snn.LIF())

ll_lstm = rb.rnn.LSTM(512, 2)

ll_ffann = rb.Sequential(rb.nn.Linear(512), rb.nn.ReLU())


LAST_LAYER = ll_ffann

model = rb.Sequential(conv_stack, LAST_LAYER, nn.Linear(N_OUT)).make_model(inp_shape)

In [3]:
model.configure(full_state=True)

In [3]:
out, h = model(example)

In [5]:

state_con = rb.base._utils.StateContainerNew(h)

In [6]:

state_con.get(lambda t:t.shape)

[[[],
  {'mem': torch.Size([8, 32, 12672])},
  [],
  {'mem': torch.Size([8, 32, 5120])},
  [],
  {'mem': torch.Size([8, 32, 3072])}],
 [[], []],
 []]

In [7]:

example.shape

torch.Size([8, 32, 1, 92, 76])

In [4]:

out.sum().backward()